In [1]:
%%bash
ls

countries.txt
microsoft-com.data
microsoft-com.info
Untitled.ipynb


In [2]:
%%bash
cat microsoft-com.info

1. Title: "Anonymous web data from www.microsoft.com"

2. Source Information
   -- Creators: Jack S. Breese, David Heckerman, Carl M. Kadie
     -- Microsoft Research, Redmond WA, 98052-6399, USA
     -- breese@microsoft.com, heckerma@microsoft.com, carlk@microsoft.com
   -- Donor: Breese, Heckerman, & Kadie
   -- Date: November 1998
 
3. Past Usage:
   -- J. Breese, D. Heckerman., C. Kadie _Empirical Analysis of
      Predictive Algorithms for Collaborative Filtering_ Proceedings
      of the Fourteenth Conference on Uncertainty in Artificial Intelligence,
      Madison, WI, July, 1998. 
	Also, expanded as Microsoft Research Technical Report MSR-TR-98-12,
	The papers are available on-line at:
             http://research.microsoft.com/users/breese/cfalgs.html
      -- Results: Used to compare various memory-based and model-based collobative
         filtering.
    - Predicted attribute: Goal was to predict what areas of the web site a
         user visited based on data on what other 

## Load the Microsoft.com data into HDFS

In [27]:
%%bash
hdfs dfs -rm -r /user/stud22/ex2
hdfs dfs -mkdir /user/stud22/ex2

22/05/18 11:00:55 INFO fs.TrashPolicyDefault: Moved: 'hdfs://bdl0.eng.tau.ac.il:8020/user/stud22/ex2' to trash at: hdfs://bdl0.eng.tau.ac.il:8020/user/stud22/.Trash/Current/user/stud22/ex2


In [28]:
%%bash
hdfs dfs -put /home/stud22/workspace/ex2/microsoft-com.data/microsoft-com.data /user/stud22/ex2/microsoft-com.data

In [43]:
%%bash
hdfs dfs -put /home/stud22/workspace/ex2/microsoft-com.data/countries.txt /user/stud22/ex2/countries.txt

In [44]:
%%bash
hdfs dfs -ls /user/stud22/ex2

Found 2 items
-rw-r--r--   3 stud22 stud22        387 2022-05-18 11:26 /user/stud22/ex2/countries.txt
-rw-r--r--   3 stud22 stud22    1491520 2022-05-18 11:01 /user/stud22/ex2/microsoft-com.data


In [30]:
%%bash
hdfs dfs -cat /user/stud22/ex2/microsoft-com.data | tail -n 20

V,42706,1001,1
V,42706,1004,1
V,42707,1008,1
V,42707,1030,1
V,42707,1009,1
V,42707,1058,1
V,42707,1004,1
V,42707,1018,1
V,42708,1008,1
V,42708,1027,1
V,42708,1123,1
V,42708,1038,1
V,42708,1026,1
V,42708,1041,1
V,42709,1001,1
V,42709,1003,1
V,42710,1035,1
V,42710,1001,1
V,42710,1018,1
V,42711,1008,1


## Parse the data according to the format specified in the .info file

In [85]:
from pyspark import SparkContext, SparkConf

In [86]:
raw_rdd = sc.textFile("hdfs:/user/stud22/ex2/microsoft-com.data")

In [87]:
lst_rdd = raw_rdd.map(lambda line: line.replace('"','').split(','))

In [88]:
lst_rdd.top(5)

[[u'V', u'42711', u'1008', u'1'],
 [u'V', u'42710', u'1035', u'1'],
 [u'V', u'42710', u'1018', u'1'],
 [u'V', u'42710', u'1001', u'1'],
 [u'V', u'42709', u'1003', u'1']]

## Given the list of countries in the file countries.txt (e.g., South Africa, Spain, Sweden, Switzerland),
filter the data to include only users that visited at least one page related to one of the given countries.

## ASSUMPTION: the header is the same as the country name

In [89]:
def is_attr(line):
    return line.startswith("A")

assert(not is_attr("V,42710,1001,1"))
assert(is_attr('A,1277,1,"NetShow for PowerPoint","/stream"'))

def filter_votes_by_countries(lst_rdd,countries_rdd):
    attr_rdd = lst_rdd.filter(lambda x: x[0]=='A')
    votes_rdd = lst_rdd.filter(lambda x: x[0]=='V')
    countries_lst = sc.broadcast(countries_rdd.collect())
    relevant_lines = attr_rdd.filter(lambda x: x[3] in countries_lst.values) # filter lines which contains countries
    relevant_ids = relevant_lines.map(lambda x: x[1])
    country_ids = sc.broadcast(relevant_ids.collect()) 
    country_votes = votes_rdd.filter(lambda x: x[2] in country_ids.value)
    return country_votes
    

In [82]:
attr_rdd = lst_rdd.filter(lambda x: x[0]=='A')
votes_rdd = lst_rdd.filter(lambda x: x[0]=='V')

In [83]:
attr_rdd.top(5)

[[u'A', u'1297', u'1', u'Central America', u'/centroam'],
 [u'A', u'1295', u'1', u'Training', u'/train_cert'],
 [u'A', u'1294', u'1', u'Bookshelf', u'/bookshelf'],
 [u'A', u'1293', u'1', u'Encarta', u'/encarta'],
 [u'A', u'1292', u'1', u'MS North Africa', u'/northafrica']]

In [84]:
votes_rdd.top(5)

[[u'V', u'42711', u'1008', u'1'],
 [u'V', u'42710', u'1035', u'1'],
 [u'V', u'42710', u'1018', u'1'],
 [u'V', u'42710', u'1001', u'1'],
 [u'V', u'42709', u'1003', u'1']]

In [45]:
countries_rdd = sc.textFile("hdfs:/user/stud22/ex2/countries.txt")

In [51]:
countries_lst = countries_rdd.collect() # cache in memory

In [60]:
relevant_lines = attr_rdd.filter(lambda x: x[3] in countries_lst) # filter lines which contains countries

In [59]:
relevant_lines.count() / float(attr_rdd.count())

0.13945578231292516

In [61]:
relevant_ids = relevant_lines.map(lambda x: x[1])

In [63]:
votes_rdd = lst_rdd.filter(lambda x: x[0]=='V')

In [74]:
country_ids = sc.broadcast(relevant_ids.collect()) 
country_votes = votes_rdd.filter(lambda x: x[2] in country_ids.value)

In [75]:
country_votes.top(10)

[[u'V', u'42708', u'1123', u'1'],
 [u'V', u'42706', u'1059', u'1'],
 [u'V', u'42698', u'1005', u'1'],
 [u'V', u'42688', u'1053', u'1'],
 [u'V', u'42665', u'1023', u'1'],
 [u'V', u'42649', u'1223', u'1'],
 [u'V', u'42647', u'1023', u'1'],
 [u'V', u'42641', u'1053', u'1'],
 [u'V', u'42637', u'1053', u'1'],
 [u'V', u'42636', u'1053', u'1']]

In [76]:
relevant_users = country_votes.map(lambda x: x[1]).distinct()

In [77]:
relevant_users.top(10)

[u'42708',
 u'42706',
 u'42698',
 u'42688',
 u'42665',
 u'42649',
 u'42647',
 u'42641',
 u'42637',
 u'42636']

In [78]:
print(relevant_users.count(),country_votes.count())

(3199, 3334)


In [79]:
# filter votes by the user ids we found 
user_ids = sc.broadcast(relevant_users.collect()) 
relevant_data = votes_rdd.filter(lambda x: x[1] in user_ids.value)

In [80]:
relevant_data.top(10)

[[u'V', u'42708', u'1123', u'1'],
 [u'V', u'42708', u'1041', u'1'],
 [u'V', u'42708', u'1038', u'1'],
 [u'V', u'42708', u'1027', u'1'],
 [u'V', u'42708', u'1026', u'1'],
 [u'V', u'42708', u'1008', u'1'],
 [u'V', u'42706', u'1059', u'1'],
 [u'V', u'42706', u'1004', u'1'],
 [u'V', u'42706', u'1003', u'1'],
 [u'V', u'42706', u'1001', u'1']]

In [81]:
relevant_data.count()

13111

## For each country, the number of users that visited a page of that country. Exclude from your
## report countries with a name longer than one word.

In [ ]:
one_word_countries = 